In [1]:
import numpy as np
import xarray as xr

from xarray.core.alignment import align
from scipy import optimize

from ensemblecovariancecomputer import EnsembleCovarianceComputer
from observationoperator import PointObserver
from kalmangaincomputer import KalmanGainComputer
from errorcomputer import OptimalErrorComputer, PracticalErrorComputer

from localizationfunctions import gaspari_cohn_univariate as gaspari_cohn

In [2]:
## Where are we working
proj_dir = '/Users/zstanley/Documents/git_repos/obs_loc_for_scda'
plot_dir = proj_dir + '/plots/optimal_loc'
my_data_dir = proj_dir + '/my_data/20151206.030000'
nb_dir = proj_dir + '/notebooks'

In [3]:
ds = xr.open_dataset(my_data_dir+'/temperature_covariances_averaged_tropical_pacific.nc')
ds = ds.sel(lon=209.5, lat=12.5)

In [4]:
enscov =  EnsembleCovarianceComputer()
enscov(ds)

/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/ensemblecovariancecomputer.py:148: UserWarning: Matrix is not positive definite. Smallest eigenvalue is -1.3284245274741694e-09
  warnings.warn('Matrix is not positive definite. Smallest eigenvalue is ' + str(min_eval))


In [5]:
enscov

EnsembleCovarianceComputer:

    Ensemble Covariance Matrices:
        status = set

 --- 
    Ensemble Size           : 20
    Number of Trials        : 1000

In [6]:
sst = PointObserver('sst')
sst(enscov)

In [7]:
sst

EnsembleCovarianceComputer:

    Name of Observation     : sst
    Which Fluid is Observed : ocn
    Which Level is Observed : 0
 --- 
    Pre-defined observation types:
        ['ast', 'sst']

In [8]:
ec = OptimalErrorComputer(enscov)
ec(sst)

/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/localizationfunctions.py:25: RuntimeWarning: divide by zero encountered in true_divide
  x = np.abs(distance)/np.abs(localization_half_width)
/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/localizationfunctions.py:25: RuntimeWarning: invalid value encountered in true_divide
  x = np.abs(distance)/np.abs(localization_half_width)
/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/errorcomputer.py:99: RuntimeWarning: divide by zero encountered in true_divide
  loc = np.divide(1, gaspari_cohn(dist.values, (np.abs(loc_rad)/2)))
/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/errorcomputer.py:122: RuntimeWarning: divide by zero encountered in true_divide
  loc = np.divide(1, loc_atten * gaspari_cohn(dist.values, (np.abs(loc_rad)/2)))


In [9]:
kg = KalmanGainComputer(sst)

In [10]:
kg.compute_error_true_K(level=ec.slice_atm)

0.06522415105137248

In [11]:
pec = PracticalErrorComputer(enscov)
pec(sst, enscov)

In [20]:
my_dict = pec.__dict__.copy()
#my_dict = ec.__dict__.copy()

In [21]:
[my_dict.pop(x) for x in ['len_atm','len_ocn', 'slice_atm', 'slice_ocn', 'num_trials']]

[127, 75, slice(0, 127, 1), slice(127, 202, 1), 1000]

In [24]:
my_dict

{'error_true_K_atm': 0.06522415105137248,
 'error_true_K_ocn': 0.10304930645398098,
 'error_unloc_atm': 0.869630457234635,
 'error_unloc_ocn': 0.2734425115220226,
 'error_practical_gcr_atm': 0.06904649343990676,
 'error_practical_gcr_ocn': 0.037611803523527346,
 'error_practical_cutoffloc_atm': 0.06618727590552041,
 'error_practical_cutoffloc_ocn': 0.037611803523527346}

In [14]:
keys = ['error_unloc_atm', 'error_unloc_ocn', 'locrad_gcr_atm', 'locrad_gcr_ocn', 'error_gcr_atm', \
            'error_gcr_ocn', 'locrad_gcra_atm', 'locrad_gcra_ocn', 'locatten_gcra_atm',\
            'locatten_gcra_ocn', 'error_gcra_atm', 'error_gcra_ocn', 'error_eorl_atm', 'error_eorl_ocn', 'error_true_K_atm', 'error_true_K_ocn']

In [ ]:
keys = ['error_true_K_atm', 'error_true_K_ocn', 'error_unloc_atm', 'error_unloc_ocn', 'error_practical_gcr_atm', 'error_practical_gcr_ocn', 'error_practical_cutoffloc_atm', 'error_practical_cutoffloc_ocn']